# Shufflenet

Implement ShuffleNet by Tensorflow2

In [56]:
import tensorflow as tf
from tensorflow.keras import Model,layers, Input

In [57]:
class ConvBNRelu(Model):

    def __init__(self, channels, kernel_size, strides):
        super(ConvBNRelu, self).__init__()
        self.conv = layers.Conv2D(channels, kernel_size, strides, padding='same', use_bias=False)
        self.bn = layers.BatchNormalization()
        self.relu = layers.ReLU()
    
    def __call__(self, inputs, training = True):
        x = self.conv(inputs)
        x = self.bn(x, training)
        x = self.relu(x)
        return x

class DepthwiseConvBNRelu(Model):
    def __init__(self, kernel_size, strides):
        super(DepthwiseConvBNRelu, self).__init__()
        self.depth_wise = layers.DepthwiseConv2D(kernel_size, strides, padding = 'same', use_bias=False)
        self.bn = layers.BatchNormalization()
    
    def __call__(self, inputs, training = True):
        x = self.depth_wise(inputs)
        x =self.bn(x, training)
        return x

class ChannelShuffle(Model):
    def __init__(self, group):
        super(ChannelShuffle, self).__init__()
        self.group = group
    
    def __call__(self, inputs):
        shape = inputs.shape
        # batch = shape[0]
        h = shape[1]
        w = shape[2]
        c = shape[3]
        # assert c % self.group == 0, 'c % group needs to be zero!'

        inputs = tf.reshape(inputs, shape=[-1, h, w, c // self.group, self.group])
        inputs = tf.transpose(inputs, [0, 1, 2, 4, 3])
        inputs = tf.reshape(inputs, shape=(-1, h, w, c))

        return inputs

class ShuffleBlock(Model):
    def __init__(self, channels, strides, split_ratio = 0.5):
        super(ShuffleBlock, self).__init__()
        self.split_ratio = split_ratio
        self.conv1 = ConvBNRelu(channels//2, 1, 1)
        self.depth_wise = DepthwiseConvBNRelu(3, strides = strides)
        self.conv2 = ConvBNRelu(channels//2, 1, 1)
        self.shuffle = ChannelShuffle(group=2)
    
    def __call__(self, inputs, training):
        # channle split
        x1, x2 = tf.split(inputs, num_or_size_splits=int(1/self.split_ratio), axis = -1)
        # conv_1*1 depthwise_3*3 conv1*1
        x2 = self.conv1(x2, training)
        x2 = self.depth_wise(x2, training)
        x2 = self.conv2(x2, training)
        # concatenate x1 and x2 to make information comunicate
        feature = layers.Concatenate()([x1, x2])
        # channel shuffle
        res = self.shuffle(feature)
        return res

class ShuffleConvBlock(Model):

    def __init__(self, in_channels, out_channels, strides):
        super(ShuffleConvBlock, self).__init__()
        self.conv1 = ConvBNRelu(out_channels - in_channels, 1, 1)
        self.depth_wise = DepthwiseConvBNRelu(3, strides=strides)
        self.conv2 = ConvBNRelu(out_channels - in_channels, 1, 1)
        self.depth_wise_lateral = DepthwiseConvBNRelu(3, strides=strides)
        self.conv_lateral = ConvBNRelu(in_channels, 1, 1)
        self.shuffle = ChannelShuffle(group=2)

    def __call__(self, inputs, training):
        x1, x2 = inputs, inputs
        x2 = self.conv1(x2, training)
        x2 = self.depth_wise(x2, training)
        x2 = self.conv2(x2, training)

        x1 = self.depth_wise_lateral(x1, training)
        x1 = self.conv_lateral(x1, training)

        feature = layers.Concatenate()([x1, x2])
        res = self.shuffle(feature)

        return res

class ShuffleNetStage(Model):

    def __init__(self, repeat, in_channels, out_channels):
        super(ShuffleNetStage, self).__init__()
        self.shuffle_conv_block = ShuffleConvBlock(in_channels=in_channels,
                                                   out_channels=out_channels,
                                                   strides=2)
        self.convs = []
        for i in range(repeat):
            self.convs.append(ShuffleBlock(channels=out_channels,
                                           strides=1))

    def __call__(self, inputs, training):
        x = self.shuffle_conv_block(inputs, training)
        for conv in self.convs:
            x = conv(x, training)

        return x

class ShuffleNetV2(Model):
    """ShuffleNetV2
    How to reduce MAC:
    1.make channels_in == channels_out
    2.don't use group convolution
    3.change add to concatenate
    4.don't make model fragmented
    So, use:
    1.conv_1X1
    2.depthwise and pointwise
    3.concatenate rather than add
    4.maybe shuffle block can promote accuracy
    """

    def __init__(self, channels=[24, 116, 232, 464, 1024]):
        super(ShuffleNetV2, self).__init__()
        self.conv1 = layers.Conv2D(channels[0], 3, 2, padding='same')
        self.pool = layers.MaxPool2D(3, strides=2, padding='same')
        self.stage1 = ShuffleNetStage(repeat=3, in_channels=channels[0], out_channels=channels[1])
        self.stage2 = ShuffleNetStage(repeat=7, in_channels=channels[1], out_channels=channels[2])
        self.stage3 = ShuffleNetStage(repeat=3, in_channels=channels[2], out_channels=channels[3])
        self.conv2 = layers.Conv2D(channels[4], kernel_size=1, padding='same')

    def __call__(self, inputs, training):
        x = self.conv1(inputs)
        x = self.pool(x)
        x = self.stage1(x, training)
        x = self.stage2(x, training)
        x = self.stage3(x, training)
        x = self.conv2(x)

        return x



In [58]:
shufflenet_v2 = ShuffleNetV2()
inputs_ = Input(shape=(224, 224, 3))
res = shufflenet_v2(inputs_, training=True)
model = Model(inputs_, res)
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_666 (Conv2D)             (None, 112, 112, 24) 672         input_17[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_77 (MaxPooling2D) (None, 56, 56, 24)   0           conv2d_666[0][0]                 
__________________________________________________________________________________________________
conv2d_667 (Conv2D)             (None, 56, 56, 92)   2208        max_pooling2d_77[0][0]           
____________________________________________________________________________________________

# SqueezeNet

Implement SqueezeNet by Tensorflow2

In [62]:
import tensorflow as tf
from tensorflow.keras import Input, Model, layers

class Mish(layers.Layer):
    def __init__(self):
        super(Mish, self).__init__()

    def __call__(self, inputs):
        return tf.multiply(inputs, tf.tanh(tf.nn.softplus(inputs)))

class FireBlock(layers.Layer):

    def __init__(self, filter1, filter2, filter3):
        super(FireBlock, self).__init__()
        self.conv1 = layers.Conv2D(filters=filter1, kernel_size=1, padding='same', use_bias=False)
        self.conv2 = layers.Conv2D(filters=filter2, kernel_size=1, padding='same', use_bias=False)
        self.conv3 = layers.Conv2D(filters=filter3, kernel_size=3, padding='same', use_bias=False)
        self.bn1 = layers.BatchNormalization()
        self.bn2 = layers.BatchNormalization()
        self.bn3 = layers.BatchNormalization()

    def __call__(self, inputs):
        squeeze_x = self.conv1(inputs)
        squeeze_x = self.bn1(squeeze_x)
        squeeze_x = Mish()(squeeze_x)
        expand_x1 = self.conv2(squeeze_x)
        expand_x1 = self.bn2(expand_x1)
        expand_x1 = Mish()(expand_x1)
        expand_x3 = self.conv3(squeeze_x)
        expand_x3 = self.bn3(expand_x3)
        expand_x3 = Mish()(expand_x3)

        merge_x = layers.Concatenate()([expand_x1, expand_x3])

        return merge_x

class SqueezeNet(Model):
    def __init__(self):
        super(SqueezeNet, self).__init__()
        self.conv1 = layers.Conv2D(filters=96, kernel_size=7, strides=2, padding='same', use_bias=False)
        self.bn1 = layers.BatchNormalization()
        self.act1 = Mish()
        self.pool1 = layers.MaxPool2D(3, strides=2, padding='same')
        self.fire_block2 = FireBlock(16, 64, 64)
        self.fire_block3 = FireBlock(16, 64, 64)
        self.fire_block4 = FireBlock(32, 128, 128)
        self.pool4 = layers.MaxPool2D(3, strides=2, padding='same')
        self.fire_block5 = FireBlock(32, 128, 128)
        self.fire_block6 = FireBlock(48, 192, 192)
        self.fire_block7 = FireBlock(48, 192, 192)
        self.fire_block8 = FireBlock(64, 256, 256)
        self.pool8 = layers.MaxPool2D(3, strides=2, padding='same')
        self.fire_block9 = FireBlock(64, 256, 256)
    
    def __call__(self, inputs = None):
        if not inputs:
            inputs = Input(shape=(224, 224, 3))
        x = self.conv1(inputs)
        x = self.bn1(x)
        x = self.act1(x)
        x = self.pool1(x)
        x = self.fire_block2(x)
        x = self.fire_block3(x)
        x = self.fire_block4(x)
        x = self.pool4(x)
        x = self.fire_block5(x)
        x = self.fire_block6(x)
        x = self.fire_block7(x)
        x = self.fire_block8(x)
        x = self.pool8(x)
        x = self.fire_block9(x)
        squeeze = Model(inputs, x)
        return squeeze
squeeze_net = SqueezeNet()
model = squeeze_net()
model.summary()
        

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_20 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_753 (Conv2D)             (None, 112, 112, 96) 14112       input_20[0][0]                   
__________________________________________________________________________________________________
batch_normalization_484 (BatchN (None, 112, 112, 96) 384         conv2d_753[0][0]                 
__________________________________________________________________________________________________
tf.math.softplus_189 (TFOpLambd (None, 112, 112, 96) 0           batch_normalization_484[0][0]    
____________________________________________________________________________________________

In [63]:
squeeze_net = SqueezeNet()
model = squeeze_net()
model.summary()

Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_21 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_778 (Conv2D)             (None, 112, 112, 96) 14112       input_21[0][0]                   
__________________________________________________________________________________________________
batch_normalization_509 (BatchN (None, 112, 112, 96) 384         conv2d_778[0][0]                 
__________________________________________________________________________________________________
tf.math.softplus_214 (TFOpLambd (None, 112, 112, 96) 0           batch_normalization_509[0][0]    
___________________________________________________________________________________________

# InceptionNet

Implement Inception Net by Tensorflow

In [3]:
import tensorflow as tf
from tensorflow.keras import Input, layers, Model

def inception_block(inputs, filter1, filter3_reduce, filter3, filter5_reduce, filter5, filter_pool, depth=2):
    x1 = layers.Conv2D(filter1, 1, padding='same', activation='relu')(inputs)

    x2 = layers.Conv2D(filter3_reduce, 1, padding='same', activation='relu')(inputs)
    x2 = layers.Conv2D(filter3, 3, padding='same', activation='relu')(x2)

    x3 = layers.Conv2D(filter5_reduce, 1, padding='same', activation='relu')(inputs)
    x3 = layers.Conv2D(filter5, 5, padding='same', activation='relu')(x3)

    x4 = layers.MaxPool2D(3, strides=1, padding='same')(inputs)
    x4 = layers.Conv2D(filter_pool, 1, padding='same', activation='relu')(x4)

    x5 = layers.Concatenate()([x1, x2, x3, x4])

    return x5

def inception_v1(inputs):
    '''Inception
        1. discard dense latyer
        2. network in network: Use conv_1X1 to reduce params and use conv_3X3, conv_5X5 to make [receptive field rich]. Make feature rich.
    '''
    x = layers.Conv2D(64, 7, 2, padding='same', activation='relu')(inputs)
    x = layers.MaxPool2D(3, 2, padding='same')(x)
    x = layers.Conv2D(64, 1, padding='same', activation='relu')(x)
    x = layers.Conv2D(192, 3, padding='same', activation='relu')(x)
    x = layers.MaxPool2D(3, 2, padding='same')(x)

    x1 = inception_block(x, 64, 96, 128, 16, 32, 32)
    x2 = inception_block(x1, 128, 128, 192, 32, 96, 64)
    x3 = layers.MaxPool2D(3, strides=2, padding='same')(x2)
    x4 = inception_block(x3, 192, 96, 208, 16, 48, 64)
    x5 = inception_block(x4, 160, 112, 224, 24, 64, 64)
    x6 = inception_block(x5, 128, 128, 256, 24, 64, 64)
    x7 = inception_block(x6, 112, 144, 288, 32, 64, 64)
    x8 = inception_block(x7, 256, 160, 320, 32, 128, 128)

    x9 = layers.MaxPool2D(3, 2, padding='same')(x8)

    x10 = inception_block(x9, 256, 160, 320, 32, 128, 128)
    x11 = inception_block(x10, 384, 192, 384, 48, 128, 128)

    x12 = layers.AvgPool2D(7, strides=1)(x11)
    x13 = layers.Dropout(rate=0.4)(x12)
    x14 = layers.Dense(1000, activation='softmax')(x13)

    return x14

img_input = Input(shape=(224, 224, 3))

outputs = inception_v1(img_input)

inception = Model(img_input, outputs)

inception.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_114 (Conv2D)             (None, 112, 112, 64) 9472        input_3[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_26 (MaxPooling2D) (None, 56, 56, 64)   0           conv2d_114[0][0]                 
__________________________________________________________________________________________________
conv2d_115 (Conv2D)             (None, 56, 56, 64)   4160        max_pooling2d_26[0][0]           
______________________________________________________________________________________________

# Mobile Net

Implement Mobile net by Tensorflow

MobileNetV1

In [14]:
import tensorflow as tf
from tensorflow.keras import Model, layers

class conv_block(Model):
    def __init__(self, filters, kernel_size=(3, 3), strides = 1):
        super().__init__()
        self._filters = filters
        self._kernel_size = kernel_size
        self._strides = strides
    
    def call(self, inputs):
        x = layers.Conv2D(filters=self._filters,kernel_size=self._kernel_size, strides=self._strides, padding='same', use_bias=False)(inputs)
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)
        return x

class depthwise_conv_block(Model):
    def __init__(self, pointwise_conv_filters, strides = 1):
        super().__init__()
        self._pointwise_conv_filters = pointwise_conv_filters
        self._strides = strides
    
    def call(self, inputs):
        x = layers.DepthwiseConv2D(kernel_size=(3, 3), strides=self._strides, padding='SAME', use_bias=False)(inputs)
        x =layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)
        x = layers.Conv2D(filters=self._pointwise_conv_filters, kernel_size=1, padding='same', use_bias=False)(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)
        return x

class mobilenet_v1(Model):
    def __init__(self, classes):
        super().__init__()
        self._classes = classes
    
    def build(self, input_shape):
        super(mobilenet_v1, self).build(input_shape)

    def call(self, inputs):
        x = conv_block(32, 2)(inputs)
        x = depthwise_conv_block(64)(x)
        x = depthwise_conv_block(128, strides=(2,2))(x)
        x = depthwise_conv_block(128)(x)
        x = depthwise_conv_block(256, strides=(2,2))(x)
        x = depthwise_conv_block(256)(x)
        x = depthwise_conv_block(512, strides=(2,2))(x)
        x = depthwise_conv_block(512)(x)
        x = depthwise_conv_block(512)(x)
        x = depthwise_conv_block(512)(x)
        x = depthwise_conv_block(512)(x)
        x = depthwise_conv_block(512)(x)
        x = depthwise_conv_block(1024, strides=(2,2))(x)
        x = depthwise_conv_block(1024)(x)
        x = layers.GlobalAveragePooling2D()(x)
        x = layers.Dense(self._classes, activation='softmax')(x)
        return x

In [19]:
import numpy as np
inputs = np.zeros((10, 32, 32, 3), dtype=np.float32)
model = mobilenet_v1(10)
model.build(input_shape=inputs.shape)
model.summary()
print(inputs.shape)

Model: "mobilenet_v1_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________
(10, 32, 32, 3)
